In [ ]:
# Installation of packages and setting up the environment
!pip install transformers
!pip install tensorboardx
!pip install simpletransformers

In [ ]:
# Installation of "apex" package from a cloned GitHub Repository so as to enable GPU utilisation

%%writefile setup.sh

export CUDA_HOME=/usr/local/cuda-10.1
git clone https://github.com/NVIDIA/apex
pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./apex

In [ ]:
!sh setup.sh

In [ ]:
# Code cell to confirm the connection with the remote GPU
import tensorflow as tf
tf.test.gpu_device_name()

In [ ]:
import pandas as pd

df = pd.read_excel(r"nlp_data.xlsx")

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test ,y_train, y_test = train_test_split(df["text"], df["label"], test_size = 0.3, random_state = 42)

In [ ]:
from simpletransformers.classification import ClassificationModel

# MODEL ARGUMENTS FOR ALL BERT MODELS

# bert, bert-base-multilingual-cased
# distilbert, distilber-base-multilingual-cased
# xlm, xlm-mlm-100-1280

model = ClassificationModel("xlm", "xlm-mlm-100-1280", num_labels = 8, use_cuda = True, args = {"overwrite_output_dir": True, 
                                                                                                "train_batch_size": 8,
                                                                                                "eval_batch_size": 8,
                                                                                                "num_train_epochs": 5,
                                                                                                "learning_rate": 4e-5})

In [ ]:
train_df = pd.concat([x_train, y_train], axis = 1)

In [ ]:
model.train_model(train_df)

In [ ]:
eval_df = pd.concat([x_test, y_test], axis = 1)

In [ ]:
import sklearn

result, model_outputs, wrong_predictions = model.eval_model(eval_df, accuracy = sklearn.metrics.accuracy_score)

In [ ]:
predictions, raw_outputs = model.predict(x_test)

In [ ]:
print(result)

In [ ]:
print(predictions)
print(raw_outputs)


In [ ]:
from sklearn.preprocessing import label_binarize
y_bin = label_binarize(y_test, classes = [1, 2, 3, 4, 5, 6, 7])
n_classes = y_bin.shape[1]

print(y_bin)

In [ ]:
import matplotlib.pyplot as plt
from itertools import cycle
from sklearn.metrics import roc_curve, roc_auc_score, auc

fpr = dict()
tpr = dict()
roc_auc = dict()

for i in range(1, 8):
    fpr[i - 1], tpr[i - 1], _ = roc_curve(y_bin[:, i - 1], raw_outputs[:, i])
    roc_auc[i - 1] = auc(fpr[i - 1], tpr[i - 1])

fig = plt.gcf()
fig.set_size_inches(18.5, 10.5)
lw = 2
colors = cycle(["aqua", "darkorange", "cornflowerblue", "deeppink", "green", "navy", "yellow"])
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color = color, lw = lw,
             label = "ROC curve of class {0} (AUC = {1:0.2f})"
             "".format(i + 1, roc_auc[i]))

plt.plot([0, 1], [0, 1], "k--", lw = lw)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Receiver Operating Characteristic curve for XLM")
plt.legend(loc = "lower right")
plt.savefig("roc_auc_bert.png")
plt.show()